In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.utils import image_dataset_from_directory

In [55]:
train_ds = image_dataset_from_directory(
    "../data/human-and-non-human/training_set/training_set",
    validation_split=0.3,
    subset="training",
    seed=123,
    image_size=(250,250),
    batch_size=32,
    labels="inferred",
)
train_ds

Found 8017 files belonging to 2 classes.
Using 5612 files for training.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 250, 250, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [54]:
val_ds = image_dataset_from_directory(
 "../data/human-and-non-human/training_set/training_set",
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(250,250),
  batch_size=32,
  labels="inferred",
)
val_ds

Found 8017 files belonging to 2 classes.
Using 2405 files for validation.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 250, 250, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [9]:
from keras import Sequential
from keras import layers
from keras.layers import Rescaling

In [56]:

from keras import layers, models

# Load pretrained MobileNetV2
base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(250, 250, 3))

# Freeze the base model (not trainable)
base_model.trainable = False

# Add custom layers on top for your specific task
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # For binary classification: human (1) vs not human (0)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Now, you can fine-tune it or train on your human vs not-human dataset
model.fit(train_ds, epochs=5, validation_data=val_ds)


C:\Users\HP\AppData\Local\Temp\ipykernel_13808\540324685.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(250, 250, 3))


Epoch 1/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 134s 741ms/step - accuracy: 0.8858 - loss: 0.2568 - val_accuracy: 0.9609 - val_loss: 0.1045
Epoch 2/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 137s 782ms/step - accuracy: 0.9701 - loss: 0.0805 - val_accuracy: 0.9696 - val_loss: 0.0745
Epoch 3/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 146s 830ms/step - accuracy: 0.9844 - loss: 0.0482 - val_accuracy: 0.9613 - val_loss: 0.0927
Epoch 4/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 119s 676ms/step - accuracy: 0.9880 - loss: 0.0369 - val_accuracy: 0.9742 - val_loss: 0.0671
Epoch 5/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 78s 441ms/step - accuracy: 0.9916 - loss: 0.0291 - val_accuracy: 0.9825 - val_loss: 0.0495


In [59]:

test_training_ds = image_dataset_from_directory(
  "../data/human-and-non-human/test_set/test_set",
  seed=123,
  image_size=(250,250),
  batch_size=32,
  labels="inferred",

)
test_training_ds

Found 2723 files belonging to 2 classes.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 250, 250, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [60]:
model.evaluate(test_training_ds)

86/86 ━━━━━━━━━━━━━━━━━━━━ 27s 308ms/step - accuracy: 0.9737 - loss: 0.0705


[0.06365882605314255, 0.977230966091156]

In [61]:
train_ds.class_names

['humans', 'non-humans']

In [71]:
import cv2 as cv
import numpy as np

# Load grayscale image
image = cv.imread("../data/human-and-non-human/sarbesh.jpg")

# Resize and normalize
resized_image = cv.resize(image, (250, 250))
resized_image = np.array(resized_image)

resized_image = np.expand_dims(resized_image , axis=0)

# Now this will work
prediction = model.predict(resized_image)
if prediction>0.5:
    print("human")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
human


In [68]:
model.predict(resized_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


array([[0.97927094]], dtype=float32)

In [73]:
import joblib
joblib.dump(model,"human_recog_model.joblib")

['human_recog_model.joblib']

In [35]:
import pandas as pd
import random


hospital_names = [
    "Amri Hospital", "City Hospital", "Medilane Hospital",
    "Care Hospital", "Sunrise Hospital", "LifeCare Hospital",
    "Apollo Hospital", "Healing Touch Clinic", "Global Health Centre",
    "Green Cross Hospital"
]


blood_groups = ["A+", "A-", "B+", "B-", "O+", "O-", "AB+", "AB-"]


urgency_levels = ["Regular Need", "URGENT"]


data = []
for _ in range(200):
    hospital = random.choice(hospital_names)
    blood_group = random.choice(blood_groups)
    urgency = random.choice(urgency_levels)
    units_needed = random.randint(1, 10)
    
    data.append([hospital, blood_group, urgency, units_needed])

df = pd.DataFrame(data, columns=["Hospital Name", "Blood Group", "Urgency", "Units Needed"])

print("Original Data:")
print(df.head())


df["Hospital Name Encoded"] = df["Hospital Name"].astype('category').cat.codes

df["Blood Group Encoded"] = df["Blood Group"].astype('category').cat.codes


df["Urgency Encoded"] = df["Urgency"].map({"Regular Need": 0, "URGENT": 1})

preprocessed_df = df[[
    "Hospital Name Encoded", "Blood Group Encoded", "Urgency Encoded", "Units Needed"
]]

print("\nPreprocessed Data (Encoded):")
print(preprocessed_df.head())

df['Shortage'] = df.apply(
    lambda row: 1 if (row['Urgency'] == "URGENT" and row['Units Needed'] > 5) else 0, 
    axis=1
)

print(df[['Blood Group', 'Urgency', 'Units Needed', 'Shortage']].head())
df


Original Data:
          Hospital Name Blood Group       Urgency  Units Needed
0       Apollo Hospital          O+        URGENT             5
1  Global Health Centre          B-  Regular Need             5
2       Apollo Hospital          O+        URGENT             6
3  Global Health Centre          O+        URGENT             9
4       Apollo Hospital          B-  Regular Need             3

Preprocessed Data (Encoded):
   Hospital Name Encoded  Blood Group Encoded  Urgency Encoded  Units Needed
0                      1                    6                1             5
1                      4                    5                0             5
2                      1                    6                1             6
3                      4                    6                1             9
4                      1                    5                0             3
  Blood Group       Urgency  Units Needed  Shortage
0          O+        URGENT             5         0
1    

,Hospital Name,Blood Group,Urgency,Units Needed,Hospital Name Encoded,Blood Group Encoded,Urgency Encoded,Shortage
0,Apollo Hospital,O+,URGENT,5,1,6,1,0
1,Global Health Centre,B-,Regular Need,5,4,5,0,0
2,Apollo Hospital,O+,URGENT,6,1,6,1,1
3,Global Health Centre,O+,URGENT,9,4,6,1,1
4,Apollo Hospital,B-,Regular Need,3,1,5,0,0
...,...,...,...,...,...,...,...,...
195,Healing Touch Clinic,AB-,Regular Need,7,6,3,0,0
196,City Hospital,B-,Regular Need,9,3,5,0,0
197,LifeCare Hospital,AB-,Regular Need,2,7,3,0,0
198,Amri Hospital,A-,URGENT,7,0,1,1,1


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
X = np.array(df[['Urgency Encoded' , 'Units Needed']])
y = np.array(df['Shortage'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


shortage_predict_model = RandomForestClassifier()
shortage_predict_model.fit(X_train, y_train)


print("Training accuracy:", shortage_predict_model.score(X_train, y_train))
print("Testing accuracy:", shortage_predict_model.score(X_test, y_test))


Training accuracy: 1.0
Testing accuracy: 1.0


In [31]:
import joblib
joblib.dump(shortage_predict_model , "shortage_predict_model.joblib")

['shortage_predict_model.joblib']

In [ ]:
import random
import pandas as pd

hospitals = {
    "AIIMS Bhubaneswar": (20.2465, 85.8189),
    "SUM Hospital": (20.2957, 85.8186),
    "KIMS Hospital": (20.2960, 85.8269),
    "AMRI Hospital": (20.2965, 85.8169),
    "Care Hospitals Bhubaneswar": (20.2918, 85.8305),
    "Apollo Hospitals Bhubaneswar": (20.2961, 85.8187)
}

blood_types = ['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-']

def generate_donor():
    hospital_name = random.choice(list(hospitals.keys()))
    base_lat, base_lon = hospitals[hospital_name]
    
    donor = {
        'age': random.randint(18, 60),
        'last_donation_hours': random.randint(0, 365*24),
        'urgency': random.randint(1, 5),
        'latitude': base_lat + random.uniform(-0.01, 0.01),
        'longitude': base_lon + random.uniform(-0.01, 0.01),
        'blood_type': random.choice(blood_types),
        'hospital_name': hospital_name
    }
    

    if donor['last_donation_hours'] > (90 * 24) and 18 <= donor['age'] <= 55:
        label = 1
    else:
        label = random.choices([0, 1], weights=[0.7, 0.3])[0]  
    donor['label'] = label
    return donor

data = [generate_donor() for _ in range(1000)]  

df = pd.DataFrame(data)

# Save to CSV
df.to_csv('../data/bbsr_donor_dataset.csv', index=False)

df


,age,last_donation_hours,urgency,latitude,longitude,blood_type,hospital_name,label
0,49,5471,4,20.301785,85.823671,B+,SUM Hospital,1
1,41,8396,1,20.300413,85.816381,B+,Apollo Hospitals Bhubaneswar,1
2,19,6879,5,20.304811,85.818713,A-,AMRI Hospital,1
3,40,5288,3,20.287322,85.821425,O-,KIMS Hospital,1
4,29,2837,3,20.245835,85.826786,O+,AIIMS Bhubaneswar,1
...,...,...,...,...,...,...,...,...
995,39,3402,3,20.290919,85.833439,B+,KIMS Hospital,1
996,25,4472,5,20.304984,85.811236,AB-,SUM Hospital,1
997,28,5450,2,20.290832,85.826657,AB-,AMRI Hospital,1
998,20,3983,2,20.304914,85.836121,B+,KIMS Hospital,1


In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


donor_dataset = pd.read_csv('../data/bbsr_donor_dataset.csv')
print(df.head())

label_encoders = {}
for col in ['blood_type', 'hospital_name']:
    le = LabelEncoder()
    donor_dataset[col] = le.fit_transform(donor_dataset[col])
    label_encoders[col] = le

X = donor_dataset.drop(columns=['label'])
y = donor_dataset['label']


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


donation_prob_model = LogisticRegression()
donation_prob_model.fit(X_train, y_train)


y_pred = donation_prob_model.predict(X_test)
print(classification_report(y_test, y_pred))


probabilities = donation_prob_model.predict_proba(X_test)[:, 1]  


print("First 5 donation probabilities:", probabilities[:5])


          Hospital Name Blood Group       Urgency  Units Needed  \
0       Apollo Hospital          O+        URGENT             5   
1  Global Health Centre          B-  Regular Need             5   
2       Apollo Hospital          O+        URGENT             6   
3  Global Health Centre          O+        URGENT             9   
4       Apollo Hospital          B-  Regular Need             3   

   Hospital Name Encoded  Blood Group Encoded  Urgency Encoded  Shortage  
0                      1                    6                1         0  
1                      4                    5                0         0  
2                      1                    6                1         1  
3                      4                    6                1         1  
4                      1                    5                0         0  
              precision    recall  f1-score   support

           0       0.70      0.43      0.53        53
           1       0.82      0.93     

In [37]:
import joblib
joblib.dump(donation_prob_model , "donation_prob_model.joblib")

['donation_prob_model.joblib']

In [38]:
donor_dataset

,age,last_donation_hours,urgency,latitude,longitude,blood_type,hospital_name,label
0,49,5471,4,20.301785,85.823671,4,5,1
1,41,8396,1,20.300413,85.816381,4,2,1
2,19,6879,5,20.304811,85.818713,1,1,1
3,40,5288,3,20.287322,85.821425,7,4,1
4,29,2837,3,20.245835,85.826786,6,0,1
...,...,...,...,...,...,...,...,...
995,39,3402,3,20.290919,85.833439,4,4,1
996,25,4472,5,20.304984,85.811236,3,5,1
997,28,5450,2,20.290832,85.826657,3,1,1
998,20,3983,2,20.304914,85.836121,4,4,1
